# Kalman filter brute force paramater search

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import statsmodels.formula.api as sm
import statsmodels.tsa.stattools as ts
import statsmodels.tsa.vector_ar.vecm as vm

import time
from IPython.display import clear_output

In [7]:
# df1 = pd.read_csv("../data/ZECUSDT-minute.csv", index_col=0, parse_dates=True)
# df2 = pd.read_csv("../data/XMRUSDT-minute.csv", index_col=0, parse_dates=True)
df1 = pd.read_csv("../data/testing-hourly/CELRUSDT-minute.csv", index_col=0, parse_dates=True)
df2 = pd.read_csv("../data/testing-hourly/FETUSDT-minute.csv", index_col=0, parse_dates=True)
# df1 = pd.read_csv("../data/testing-hourly/BTTUSDT-minute.csv", index_col=0, parse_dates=True)
# df2 = pd.read_csv("../data/testing-hourly/HOTUSDT-minute.csv", index_col=0, parse_dates=True)

print(len(df1))
print(len(df2))

1212637
1247928


In [8]:
df = df1.close.rename("A").to_frame()
df["B"] = df2.close
df = df[4000:]
df = df.dropna()
results = sm.ols(formula="B ~ A", data=df[['B', 'A']]).fit()
hedgeRatio = results.params[1]

In [14]:
df = df[-50000:]

In [15]:
def get_result(lm, ls, thres, s_thres, fee=0.003):
    """generates the profit
    l is length/lookback period, used for spread's mean and std"""
    spread = pd.Series((df['B'] - hedgeRatio * df['A'])).rename("spread").to_frame()
    spread["mean"] = spread.spread.rolling(lm).mean()
    spread["std"] =  spread.spread.rolling(ls).std()
    spread["zscore"] = pd.Series((spread["spread"]-spread["mean"])/spread["std"])
    spread = spread.dropna()
    spread["A"] = df1["close"].reindex(spread.index)
    spread["Ah"] = df1["high"].reindex(spread.index)
    spread["Al"] = df1["low"].reindex(spread.index)

    spread["B"] = df2["close"].reindex(spread.index)
    spread["Bh"] = df2["high"].reindex(spread.index)
    spread["Bl"] = df2["low"].reindex(spread.index)
    return run_backtest(spread, thres, s_thres, fee)
    
def run_backtest(spread, thres, s_thres, fee):
    # Buy  random value between high and low
    total = 100.
    fee = fee       #Set double normal rate
    thres = thres         #2
    sell_thres = s_thres   # 0, in range [0, ...) for more travel
    price_a = None
    price_b = None
    long = None       #Values: None, "A", "B"
    trades = 0
    for i in range(spread.shape[0]):
        z = spread.zscore[i]
        if long == None: # Looking to buy
            if z>thres or z<-thres:
                trades += 1
                al, ah, bl, bh = get_a_b(spread.Al[i], spread.A[i], spread.Ah[i], spread.Bl[i], spread.B[i], spread.Bh[i])
                if z>thres: 
                    price_a = [ah, ((total/2)/ah)*(1.-fee)]
                    price_b = [bl, ((total/2)/bl)*(1.-fee)]
                    long = "B"
                elif z<-thres:
                    price_a = [al, ((total/2)/al)*(1.-fee)]
                    price_b = [bh, ((total/2)/bh)*(1.-fee)]
                    long = "A"
        if long == "A" and z>sell_thres: #Liquidate positions
            al, ah, bl, bh = get_a_b(spread.Al[i], spread.A[i], spread.Ah[i], spread.Bl[i], spread.B[i], spread.Bh[i])
            total += (price_a[0] - ah)*price_a[1]-(ah*price_a[1]*fee)
            total += (bl - price_b[0])*price_b[1]-(bl*price_b[1]*fee)
            price_a, price_b, long = None, None, None
        elif long == "B" and z<-sell_thres: #Liquidate positions
            al, ah, bl, bh = get_a_b(spread.Al[i], spread.A[i], spread.Ah[i], spread.Bl[i], spread.B[i], spread.Bh[i])
            total += (al - price_a[0])*price_a[1]-(al*price_a[1]*fee)
            total += (price_b[0] - bh)*price_b[1]-(bh*price_b[1]*fee)
            price_a, price_b, long = None, None, None
    return total, trades
    
def get_a_b(al, ac, ah, bl, bc, bh):
    return ac-abs(ac-al)/2, ac+abs(ac-ah)/2, bc-abs(bc-bl)/2, bc+abs(bc-bh)/2

In [16]:
# l              1000.000000
# thres             1.300000
# sell_thres        1.500000
# trades         1732.000000
# result        10120.093557
# Name: 151, dtype: float64

In [17]:
d = {"l":[], "thres":[], "sell_thres":[], "trades":[], "result":[]}
for l in [500, 1000, 2000, 4000, 6000, 8000, 10000, 13000, 15000, 20000, 30000, 50000]: #ADD 500, 1000
    for thres in [0.4, 0.7, 1, 1.3, 1.7, 2., 2.3, 2.6, 2.9, 3.2]:
        for sell_thres in [-1.8, -1.5, -1.2, -0.9, -0.6, -0.3, 0., 0.3, 0.6, 0.9, 1.2, 1.5, 2.]:
            if sell_thres <= -thres:
                continue
            print(f"Now doing l: {l}, thres: {thres}, sell_thres: {sell_thres}, safe to kill kernel")
            result, trades = get_result(l, l, thres, sell_thres)
            print(f"DON'T KILL KERNEL, ADDING TRADES")
            time.sleep(1)
            d["trades"].append(trades)
            d["result"].append(result)
            d["l"].append(l)
            d["thres"].append(thres)
            d["sell_thres"].append(sell_thres)
            clear_output()
            print(pd.DataFrame(d).sort_values("result", ascending=False).iloc[0])
                

l             6000.000000
thres            0.400000
sell_thres       0.900000
trades          23.000000
result         129.791698
Name: 464, dtype: float64


In [14]:
df = pd.DataFrame(d)

In [15]:
df.sort_values("result", ascending=False).head(20)

,l,thres,sell_thres,trades,result
233,2000,0.4,0.6,2548,2.561568e+06
242,2000,0.7,0.6,2243,1.513237e+06
128,1000,0.7,0.9,2850,8.085268e+05
243,2000,0.7,0.9,1504,7.602887e+05
151,1000,1.3,1.5,1593,6.977237e+05
138,1000,1.0,0.9,2634,6.512699e+05
252,2000,1.0,0.6,1773,6.365216e+05
234,2000,0.4,0.9,1521,6.113817e+05
119,1000,0.4,0.9,2904,6.052345e+05
140,1000,1.0,1.5,1610,5.667849e+05


In [20]:
d = {"l":[], "thres":[], "sell_thres":[], "trades":[], "result":[]}
for l in [1400, 1600, 1800, 2000, 2200, 2400, 2600]:
    for thres in [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.]:
        for sell_thres in [0.5, 0.6, 0.7, 0.8, 0.9, 1.]:
            if sell_thres <= -thres:
                continue
            print(f"Now doing l: {l}, thres: {thres}, sell_thres: {sell_thres}, safe to kill kernel")
            result, trades = get_result(l, l, thres, sell_thres)
            print(f"DON'T KILL KERNEL, ADDING TRADES")
            time.sleep(1)
            d["trades"].append(trades)
            d["result"].append(result)
            d["l"].append(l)
            d["thres"].append(thres)
            d["sell_thres"].append(sell_thres)
            clear_output()
            print(pd.DataFrame(d).sort_values("result", ascending=False).iloc[0])
                

l             1.800000e+03
thres         6.000000e-01
sell_thres    8.000000e-01
trades        1.950000e+03
result        3.179701e+06
Name: 135, dtype: float64


In [23]:
df = pd.DataFrame(d)

In [24]:
df.sort_values("result", ascending=False).head(20)

,l,thres,sell_thres,trades,result
135,1800,0.6,0.8,1950,3.179701e+06
141,1800,0.7,0.8,1919,3.099943e+06
140,1800,0.7,0.7,2188,2.942029e+06
134,1800,0.6,0.7,2252,2.883720e+06
181,2000,0.5,0.6,2498,2.835383e+06
129,1800,0.5,0.8,1960,2.771383e+06
128,1800,0.5,0.7,2298,2.727635e+06
175,2000,0.4,0.6,2548,2.561568e+06
123,1800,0.4,0.8,1972,2.531659e+06
117,1800,0.3,0.8,1981,2.411582e+06


In [29]:
d = {"lm":[], "ls":[], "thres":[], "sell_thres":[], "trades":[], "result":[]}
for lm in [1600, 1700, 1800, 1900, 2000]:
    for ls in [1600, 1700, 1800, 1900, 2000]:
        for thres in [0.6, 0.7, 0.8]:
            for sell_thres in [0.6, 0.7, 0.8]:
                if sell_thres <= -thres:
                    continue
                print(f"Now doing lm: {lm}, ls: {ls}, thres: {thres}, sell_thres: {sell_thres}, safe to kill kernel")
                result, trades = get_result(lm, ls, thres, sell_thres)
                print(f"DON'T KILL KERNEL, ADDING TRADES")
                time.sleep(1)
                d["trades"].append(trades)
                d["result"].append(result)
                d["lm"].append(lm)
                d["ls"].append(ls)
                d["thres"].append(thres)
                d["sell_thres"].append(sell_thres)
                clear_output()
                print(pd.DataFrame(d).sort_values("result", ascending=False).iloc[0])


lm            1.800000e+03
ls            1.900000e+03
thres         6.000000e-01
sell_thres    7.000000e-01
trades        2.187000e+03
result        3.226238e+06
Name: 118, dtype: float64


In [30]:
df = pd.DataFrame(d)

In [31]:
df.sort_values("result", ascending=False).head(20)

,lm,ls,thres,sell_thres,trades,result
118,1800,1900,0.6,0.7,2187,3.226238e+06
110,1800,1800,0.6,0.8,1950,3.179701e+06
113,1800,1800,0.7,0.8,1919,3.099943e+06
127,1800,2000,0.6,0.7,2110,2.980127e+06
112,1800,1800,0.7,0.7,2188,2.942029e+06
109,1800,1800,0.6,0.7,2252,2.883720e+06
95,1800,1600,0.7,0.8,2055,2.877989e+06
121,1800,1900,0.7,0.7,2113,2.836844e+06
74,1700,1900,0.6,0.8,1924,2.686988e+06
172,1900,2000,0.6,0.7,2061,2.661213e+06
